#   `Функции потерь`

## Среднеквадратичная ошибка и задача обучения:
$$
Q(w, X, y) = \frac{1}{\ell} \sum_{i=1}^{\ell}(\langle{w,x_i}\rangle - y_i)^2 \rightarrow min 
$$
где $x_i$ — это $i$-ый объект датасета, $y_i$ — правильный ответ для $i$-го объекта, а $w$ — веса нашей линейной модели.

Как мы помним, для линейной модели, его можно записать в матричном виде вот так:

$$
Q(w, X, y) = \frac{1}{\ell} || Xw - y ||^2 \rightarrow min
$$
где $X$ — это матрица объекты-признаки, а $y$ — вектор правильных ответов.

Если не помним:
$$
|| z || = \sqrt{\sum_{j=1}^{n} z_j^2}
$$
*Евклидова норма

Характеристики:
1. Из-за квадрата отклонения очень чувствительна к выбросам
2. Популярна
3. Применяется в случаях, когда требуется подчеркнуть большие ошибки и выбрать модель, которая дает меньше именно больших ошибок.

В коде:

In [7]:
import numpy as np

X = np.arange(200).reshape(20, 10)
y = np.arange(20)
w = np.arange(10)

def calc_mse(X: np.ndarray, y: np.ndarray, w: np.ndarray) -> float:
    
    mse = np.square(np.dot(X,w) - y).mean()
    return mse

print(calc_mse(X, y, w))
#https://artofproblemsolving.com/wiki/index.php/LaTeX:Symbols

27410283.5


# RMSE

Квадратный корень из MSE.
$$
RMSE(w, X, y) = \sqrt{\frac{1}{\ell} \sum_{i=1}^{\ell}(\langle{w,x_i}\rangle - y_i)^2} \rightarrow min 
$$
В коде:

In [13]:
def calc_rmse(X,y,w) -> float:
    
    rmse = np.sqrt(np.square(np.dot(X,w) - y).mean())
    return rmse

print(calc_rmse(X,y,w))

5235.483120018629


## Средняя абсолютная ошибка

$$
Q(a, X) = \frac{1}{\ell} \sum_{i=1}^{\ell}| a(x_i) - y_i | \rightarrow min
$$

Пояснение:

$a(x) = \langle{w,x}\rangle$

Характеристики:
1. Не так чувствительна к выбросам как MSE.
2. Я бы сказал на втором месте по популярности.
3. Все ошибки в среднем взвешены одинаково. 

В коде:

In [8]:
def calc_mae(X: np.ndarray, y: np.ndarray, w: np.ndarray) -> float:
    
    mae = np.abs((np.dot(X,w) - y).mean())
    return mae

print(calc_mae(X, y, w))


4550.5


## $R^2$ или R2 score или Коэффициент детерминации

$$
R^2 = 1 - \frac{\frac{1}{\ell} \sum_{i}^{\ell}(y - \hat{y})^2}{\frac{1}{\ell} \sum_{i}^{\ell}(y - \bar{y})^2}
$$
$
Числитель:\frac{1}{\ell} \sum_{i}^{\ell}(y - \hat{y})^2 - MSE
$

$
Знаменатель:\frac{1}{\ell} \sum_{i}^{\ell}(y - \bar{y})^2 - дисперсия
$

В коде:

In [9]:
def r2(X: np.ndarray, y: np.ndarray, w: np.ndarray) -> float:
    
    y_variance = sum(y)/len(y)
    mse = np.square(y - (np.dot(X, w))).mean()
    dispersion = np.square(y - y_variance).mean()
    r2 = 1 - mse/dispersion
    return r2

print(r2(X,y,w))

-824368.4285714285


## Функция потерь Хубера

$$
L_H(y,a) = \begin{cases} \frac{1}{2}(y-a)^2 , & |y -a| < \delta \\ \delta(|y - a| - \frac{1}{2}\delta), & |y - a| \ge \delta \end{cases}
$$

Функционал ошибки:

$$
Q(a, X) = \frac{1}{\ell} \sum_{i=1}^{\ell}L_H(y_i, a(x_i))
$$

Пояснение:

Хуберт по сути совмещает две предыдущие функции, но учитывает при этом особенности этих функций. Как мы видим, MSE он использует при условии, что отлонение строго меньше некоего порога $\delta$, что помогает избежать полета в космос из-за выбросов. MAE же он использует когда отклонение больше или равно значению порога.

In [10]:
def calc_hubert_loss(X : np.ndarray, y : np.ndarray, w : np.ndarray) -> float:
    
    delta = float(input('Введите коэффициент дельта: '))
    huber_mse = 0.5*(y - np.dot(X,w))**2
    huber_mae = delta * (np.abs(y - np.dot(X,w)) - 0.5 * delta)
    huber_loss = (np.where(np.abs(y - np.dot(X,w)) <= delta, huber_mse, huber_mae)).mean()
    return huber_loss

print(calc_hubert_loss(X, y, w))

4550.0


In [11]:
class Loss_functons:
    
    """Класс, содержащий базовые функции потерь.

        Атрибуты:

        param X : np.ndarray размера (n_objects, n_features) с объектами датасета
        param y : np.ndarray размера (n_objects,) с правильными ответами
        param w : np.ndarray размера (n_features,) с весами линейной регрессии
        
        return : число -- значения функции потерь"""

    def __init__(self, X, y, w) -> None:
        self.X = X
        self.y = y
        self.w = w
    
    def calc_mse(self) -> float:
       
        """Функция среднеквадратичной ошибки.
                
            Атрибуты:
            
            param X : np.ndarray размера (n_objects, n_features) с объектами датасета
            param y : np.ndarray размера (n_objects,) с правильными ответами
            param w : np.ndarray размера (n_features,) с весами линейной регрессии
            
            return : число -- значения функции потерь"""

        mse = np.square(np.dot(X,w) - y).mean()
        return mse
    
    def calc_rmse(self) -> float:

        """Функция корня среднеквадратичной ошибки.
                
            Атрибуты:
            
            param X : np.ndarray размера (n_objects, n_features) с объектами датасета
            param y : np.ndarray размера (n_objects,) с правильными ответами
            param w : np.ndarray размера (n_features,) с весами линейной регрессии
            
            return : число -- значения функции потерь"""

        rmse = np.sqrt(np.square(np.dot(X,w) - y).mean())
        return rmse
    
    def calc_mae(self) -> float:
        
        """Функция средней абсолютной ошибки.
                
            Атрибуты:
            
            param X : np.ndarray размера (n_objects, n_features) с объектами датасета
            param y : np.ndarray размера (n_objects,) с правильными ответами
            param w : np.ndarray размера (n_features,) с весами линейной регрессии
            
            return : число -- значения функции потерь"""
        
        mae = np.abs((np.dot(X,w) - y).mean())
        return mae
    
    def r2(X, y, w):
    
        """Функция подсчета коэффициента детерминации.
                    
            Атрибуты:
                
            param X : np.ndarray размера (n_objects, n_features) с объектами датасета
            param y : np.ndarray размера (n_objects,) с правильными ответами
            param w : np.ndarray размера (n_features,) с весами линейной регрессии
                
            return : число -- значения функции потерь"""

        y_variance = sum(y)/len(y)
        mse = np.square(y - (np.dot(X, w))).mean()
        dispersion = np.square(y - y_variance).mean()
        r2 = 1 - mse/dispersion
        return r2
    
    def calc_hubert_loss(self) -> float:
        
        """Функция потерь Хубера.
                
            Атрибуты:
            
            param X : np.ndarray размера (n_objects, n_features) с объектами датасета
            param y : np.ndarray размера (n_objects,) с правильными ответами
            param w : np.ndarray размера (n_features,) с весами линейной регрессии
            param delta : float, значение порога, вводится внутри ф-ции
            
            return : число -- значения функции потерь"""
        
        delta = float(input('Введите коэффициент дельта: '))
        huber_mse = 0.5*(y - np.dot(X,w))**2
        huber_mae = delta * (np.abs(y - np.dot(X,w)) - 0.5 * delta)
        huber_loss = (np.where(np.abs(y - np.dot(X,w)) <= delta, huber_mse, huber_mae)).mean()
        return huber_loss
    
X = np.arange(200).reshape(20,10)
y = np.arange(20)
w = np.arange(10)

func1 = Loss_functons(X, y, w)
print(func1.calc_hubert_loss())

4550.0
